# Mercadolivre Scraping

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [32]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time

In [57]:
from googletrans import Translator, constants
translator = Translator()

In [2]:
url = 'https://lista.mercadolivre.com.br/sabonetes-em-barra#D[A:sabonetes%20em%20barra]'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'lxml')

In [22]:
product = soup.find_all('li', class_ = 'ui-search-layout__item')[6]

In [24]:
prod_url = product.find('div', class_ = 'andes-card andes-card--flat andes-card--default ui-search-result ui-search-result--core andes-card--padding-default andes-card--animated').find('a').get('href')

In [25]:
product_response = requests.get(prod_url)
product_soup = BeautifulSoup(response.content, 'lxml')

In [33]:
driver = webdriver.Chrome('/Users/guopinjie/Downloads/chromedriver')
driver.get(prod_url)

In [60]:
name = translator.translate(driver.find_elements_by_xpath("//h1[@class='ui-pdp-title']")[0].text.strip()).text

In [62]:
price = float(driver.find_elements_by_xpath("//meta[@itemprop='price']")[0].get_attribute('content'))

In [63]:
try:
    new_or_not = translator.translate(driver.find_elements_by_xpath("//span[@class='ui-pdp-subtitle']")[0].text.split('  |  ')[0]).text
    sold_number = int(driver.find_elements_by_xpath("//span[@class='ui-pdp-subtitle']")[0].text.split('  |  ')[1].split(' ')[0])
except:
    pass

In [78]:
dict_ = {}
for item in driver.find_elements_by_xpath("//table[@class='andes-table']/tbody/tr"):
    dict_[translator.translate(item.find_elements_by_xpath(".//th")[0].text).text] = translator.translate(item.find_elements_by_xpath(".//td")[0].text).text
dict_

{'Brand': 'Dove',
 'Line': 'Original',
 'Units per package': '6',
 'Unit weight': '90 g'}

In [81]:
description = translator.translate(driver.find_elements_by_xpath("//div[@class='ui-pdp-description']")[0].text).text

In [89]:
questions = []
answers = []
try:
    for i in driver.find_elements_by_xpath("//div[@class='ui-pdp-questions__questions-list__item-questions--others-questions']"):
        questions.append(translator.translate(i.find_elements_by_xpath(".//div")[0].text).text)
        answers.append(translator.translate(i.find_elements_by_xpath(".//div")[1].text).text)
except:
    pass

In [94]:
review_rating = float(driver.find_elements_by_xpath("//h2[@class='ui-pdp-reviews__rating__summary__average']")[0].text)

In [102]:
reviews = []
try:
    for i in driver.find_elements_by_xpath("//div[@class='ui-pdp-reviews__comments__review-comment']"):
        reviews.append(translator.translate(' '.join(i.text.split('\n')[:-1])).text)
except:
    pass

In [107]:
store = driver.find_elements_by_xpath("//div[@class='ui-box-component ui-box-component-pdp__visible--desktop']")[0]
translator.translate(store.text).text.split('\n')[0] = translator.translate(store.text).text.split('\n')[1]
store_sales_last_4_month = translator.translate(store.text).text.split('\n')[3]

In [142]:
def get_info(prod_url):
    product_dict = {}
    driver.get(prod_url)
    name = translator.translate(driver.find_elements_by_xpath("//h1[@class='ui-pdp-title']")[0].text.strip()).text
    product_dict['name'] = name
    price = float(driver.find_elements_by_xpath("//meta[@itemprop='price']")[0].get_attribute('content'))
    product_dict['price'] = price
    try:
        new_or_not = translator.translate(driver.find_elements_by_xpath("//span[@class='ui-pdp-subtitle']")[0].text.split('  |  ')[0]).text
        sold_number = int(driver.find_elements_by_xpath("//span[@class='ui-pdp-subtitle']")[0].text.split('  |  ')[1].split(' ')[0])
        product_dict['new_or_not'] = new_or_not
        product_dict['sold_number'] = sold_number
    except:
        pass
    dict_ = {}
    for item in driver.find_elements_by_xpath("//table[@class='andes-table']/tbody/tr"):
        dict_[translator.translate(item.find_elements_by_xpath(".//th")[0].text).text] = translator.translate(item.find_elements_by_xpath(".//td")[0].text).text
    product_dict.update(dict_)
    description = translator.translate(driver.find_elements_by_xpath("//div[@class='ui-pdp-description']")[0].text).text
    product_dict['description'] = description
    try:
        questions = []
        answers = []
        for i in driver.find_elements_by_xpath("//div[@class='ui-pdp-questions__questions-list__item-questions--others-questions']"):
            questions.append(translator.translate(i.find_elements_by_xpath(".//div")[0].text).text)
            answers.append(translator.translate(i.find_elements_by_xpath(".//div")[1].text).text)
        product_dict['questions'] = questions
        product_dict['answers'] = answers
    except:
        pass
    try:
        review_rating = float(driver.find_elements_by_xpath("//h2[@class='ui-pdp-reviews__rating__summary__average']")[0].text)
        product_dict['review_rating'] = review_rating
    except:
        pass
    try:
        reviews = []
        for i in driver.find_elements_by_xpath("//div[@class='ui-pdp-reviews__comments__review-comment']"):
            reviews.append(translator.translate(' '.join(i.text.split('\n')[:-1])).text)
        product_dict['reviews'] = reviews
    except:
        pass
    store = driver.find_elements_by_xpath("//div[@class='ui-box-component ui-box-component-pdp__visible--desktop']")[0]
    store_name = translator.translate(store.text).text.split('\n')[1]
    product_dict['store_name'] = store_name
    try:
        store_sales_last_4_month = translator.translate(store.text).text.split('\n')[3]
        product_dict['store_sales_last_4_month'] = store_sales_last_4_month
    except:
        pass
    return product_dict

In [134]:
all_products_list = []
url = 'https://lista.mercadolivre.com.br/sabonetes-em-barra#D[A:sabonetes%20em%20barra]'
i = 0

In [ ]:
driver = webdriver.Chrome('/Users/guopinjie/Downloads/chromedriver')

In [144]:
while True:
    i = i + 1
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'lxml')
    all_products = soup.find_all('li', class_ = 'ui-search-layout__item')
    for product in all_products:
        try:
            prod_url = product.find('div', class_ = 'andes-card andes-card--flat andes-card--default ui-search-result ui-search-result--core andes-card--padding-default andes-card--animated').find('a').get('href')
        except:
            pass
        all_products_list.append(get_info(prod_url))
    print(i)
    try:
        url = soup.find('li',class_ = 'andes-pagination__button andes-pagination__button--next').find('a').get('href')
    except:
        break

11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42


In [145]:
len(all_products_list)

2157

In [146]:
bar_soap_Brazil = pd.DataFrame(all_products_list)

In [171]:
bar_soap_Brazil = bar_soap_Brazil.drop(columns=['', 'Modelo', 'Marca', 'Peso da unidade', 'Unidades por embalagem', 'Linha', 'questions', 'answers', 'reviews', 'store_sales_last_4_month'])

In [176]:
bar_soap_Brazil = bar_soap_Brazil.drop_duplicates()

In [178]:
bar_soap_Brazil.reset_index(inplace = True, drop = True)

In [179]:
bar_soap_Brazil

,name,price,Brand,Line,Unit weight,Model,description,store_name,new_or_not,sold_number,Units per package,review_rating
0,Pack 4 Lux Bar Soaps Bouquet of Jasmine Botani...,4.90,Lux,Bouquet of Jasmine Botanicals,85 g,Bouquet of Jasmine,description\nPack 4 Lux Glycerin Soaps Bar Jas...,Your Grocery Store,NaN,NaN,NaN,NaN
1,Dove Original Soap Bar 90g,2.90,Dove,Original,90 g,Original,description\nBarra Dove Original Soap 90g Ther...,Dove,NaN,NaN,NaN,NaN
2,Pack 4 Lux Bar Soaps Bouquet of Jasmine Botani...,4.90,Lux,Bouquet of Jasmine Botanicals,85 g,Bouquet of Jasmine,description\nPack 4 Lux Glycerin Soaps Bar Jas...,Lux,NaN,NaN,NaN,NaN
3,Dove Original Soap Bar 90g,2.90,Dove,Original,90 g,Original,description\nBarra Dove Original Soap 90g Ther...,Your Grocery Store,NaN,NaN,NaN,NaN
4,Kit 6 Soap Rexona Antibacterial Deep Cleaning 84g,6.21,Rexona,Antibacterial,84 g,Deep Cleaning,description\nKit 6 Antibacterial Deep Soap Bar...,Your Grocery Store,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2129,Calendula Soap 110g X 24 - Cheiro D'ervas,66.90,NaN,Sabonete,NaN,NaN,description\nWHY BUY AT W3 LIFE?\n\nIMMEDIATE ...,Localização,NaN,NaN,NaN,NaN
2130,Sabonete De Copaíba 110g X 24 - Cheiro D'ervas,66.90,Smell of Herbs,NaN,110 g,NaN,Descrição\nPOR QUE COMPRAR NA W3 LIFE?\n\nENVI...,Localização,NaN,NaN,24,NaN
2131,Sabonete Hidratante De Juá 110g X 2 - Cheiro D...,11.90,NaN,NaN,NaN,NaN,description\nWHY BUY AT W3 LIFE?\n\nIMMEDIATE ...,Location,NaN,NaN,2,NaN
2132,Aroma Bar Soap 110g X 2 - Herbal Smell,11.90,Smell of Herbs,NaN,110 g,NaN,description\nWHY BUY AT W3 LIFE?\n\nIMMEDIATE ...,Location,NaN,NaN,2,NaN


In [292]:
bar_soap_Brazil.to_csv('Brazil_Bar_Soap.csv')

In [181]:
bar_soap_Brazil['Units per package'].isnull().sum()

933

In [163]:
all_product_names = []
all_product_questions = []
all_product_answers = []
for product in all_products_list:
    if product.get('questions'):
        all_product_names.extend([product['name']] * len(product['questions']))
        all_product_questions.extend(product['questions'])
        all_product_answers.extend(product['answers'])
question_and_answer = pd.DataFrame(list(zip(all_product_names, all_product_questions, all_product_answers)), columns = ['Name', 'Question', 'Answer'])

In [164]:
question_and_answer

,Name,Question,Answer
0,Kit 500 Soap 10 Gr Floral Hotel Pousada Free S...,I already made the purchase ... I didn't know ...,We will issue it manually and send it by email...
1,Kit 500 Soap 10 Gr Floral Hotel Pousada Free S...,"Hello, I just made the purchase of this produc...","Good Morning! The issuance is automatic, befor..."
2,Kit 500 Soap 10 Gr Floral Hotel Pousada Free S...,Could this kit be Eva's sweet?,"The fennel is in the 13 gram model, follow the..."
3,Kit 500 Soap 10 Gr Floral Hotel Pousada Free S...,"good night, what is the essence (smell)?","Good night! It has floral fragrance, we have o..."
4,Kit 500 Soap 10 Gr Floral Hotel Pousada Free S...,delivery and by the post office?,Good afternoon! Delivery is made by carrier. W...
...,...,...,...
3753,Kit With 12 Copaíba Soaps 110g - Cheiro D'ervas,"Bom dia, conseguiu checar se e de copaíba puro ?","It is pure copaiba, yes, but it contains other..."
3754,Kit With 12 Copaíba Soaps 110g - Cheiro D'ervas,"Boa noite, e de copaíba puro ?","Good night, we will confirm and we will return..."
3755,5 Box Natura Tododia Soap Bar With 5 Units 90g,Good Morning! what fragrances are available to...,Bom dia.. frutas vermelhas Cereja e avelã Noz ...
3756,5 Box Natura Tododia Soap Bar With 5 Units 90g,Jabuticaba pepper red fruits macadamia lime fr...,"Good morning, I didn't understand."


In [293]:
question_and_answer.to_csv('Brazil_Bar_Soap_question_and_answer.csv')

In [165]:
all_product_names = []
all_product_reviews = []
for product in all_products_list:
    if product.get('reviews'):
        all_product_names.extend([product['name']] * len(product['reviews']))
        all_product_reviews.extend(product['reviews'])
reviews = pd.DataFrame(list(zip(all_product_names, all_product_reviews)), columns = ['Name', 'Review'])

In [166]:
reviews

,Name,Review
0,Kit 500 Soap 10 Gr Floral Hotel Pousada Free S...,Good The product is good though. It could be o...
1,Kit 500 Soap 10 Gr Floral Hotel Pousada Free S...,Excellent Great. Excellent product. I recommen...
2,Kit 500 Soap 10 Gr Floral Hotel Pousada Free S...,Excellent Adore faser buys with you 10 note vo...
3,Kit 500 Soap 10 Gr Floral Hotel Pousada Free S...,Good The product is good though. It could be o...
4,Kit 500 Soap 10 Gr Floral Hotel Pousada Free S...,Excellent Great. Excellent product. I recommen...
...,...,...
717,Soapex Sabonete Antisséptico Proteção Diária 80g,Melhor Sabonete da Categoria! É o melhor produ...
718,Soapex Sabonete Antisséptico Proteção Diária 80g,I recommend! Best soap for hygiene and protect...
719,Soapex Sabonete Antisséptico Proteção Diária 80g,Excellent I have been using this soapex for a ...
720,Soapex Sab Antisséptico Prot Antisséptica 1% T...,Best product on the market !!! Best product fo...


In [294]:
reviews.to_csv('Brazil_Bar_Soap_reviews.csv')

## Model

In [223]:
model = [translator.translate(i).text for i in bar_soap_Brazil['Model'].dropna()]

In [200]:
from wordcloud import WordCloud, STOPWORDS 
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import string

In [227]:
delete_words = ['soap', 'grams', 'units', 'gramas', 'barra', 'male']
import spacy
# set up nlp environment in spacy
nlp = spacy.load("en_core_web_sm")
# use spacy to identify each chunk in the review after covid and put them into the chunk list
chunk_list = []
for text in model:
    doc = nlp(text.lower())
    for chunk in doc.noun_chunks:
        chunk = ' '.join([word for word in str(chunk.text).split() if word not in STOPWORDS])
        if not (len(chunk.split(' ')) == 0 or len(chunk.split(' ')) == 1):
            chunk_list.append(chunk)
# create chunk dict to record each chunk's appearence time
chunk_dict = dict()
for i in chunk_list:
    if not set.intersection(set(i.lower().split(' ')), set(delete_words)):
        if chunk_dict.get(i):
            chunk_dict[i] += 1
        else:
            chunk_dict[i] = 1
# sort chunk dict based on the value
sort_list = sorted(chunk_dict.items(), key=lambda x: x[1], reverse=True)

In [228]:
sort_list

[('vegan line', 13),
 ('deep cleaning', 5),
 ('vegan herbs line', 4),
 ('nutri protect', 4),
 ('biocrema capim', 3),
 ('baby original', 2),
 ('neutrogerm antiseptic body', 2),
 ('dermotivin control', 2),
 ('secret nature pitaya', 2),
 ('granado bebê dermocalmante peles sensíveis - sabonete', 2),
 ('protex oats', 2),
 ('oil control', 2),
 ('organic pure vegetable white tea', 2),
 ('soapex sab antiseptic prot', 2),
 ('dermotivin salix', 2),
 ('bioderma atoderm', 2),
 ('dermotivin original sab', 2),
 ('facial foam 130ml', 2),
 ('verisol wrinkle', 2),
 ('hydrolyzed powder light', 2),
 ('balanced hydration', 1),
 ('condicionador vegano', 1),
 ('linha vegan', 1),
 ('limpeza profunda', 1),
 ('vegan flowers line', 1),
 ('lavender officinale', 1),
 ('anti acne', 1),
 ('sab facial foam', 1),
 ('vanilla 6x90g', 1),
 ('granado glicerina frutas brasileiras - sabonete', 1),
 ('healthy hydration', 1),
 ('florentine hills', 1),
 ('granado terrapeutics castanha', 1),
 ('de glicerina - sabonete', 1),
 (

In [229]:
# draw bubble chart for the most ten frequency chunks
import plotly.graph_objects as go
x_data = list(range(10))
y_data = list(zip(*sort_list))[1][:10]
z_data = list(zip(*sort_list))[0][:10]
annotations = [
    dict(
        x=x_data[i], 
        y=y_data[i],
        text=z_data[i], # Some conditional to define outliers
        showarrow=False,
        xanchor='center',  # Position of text relative to x axis (left/right/center)
        yanchor='top' if i%2 else 'bottom',  # Position of text relative to y axis (top/bottom/middle)
    ) for i in range(10)
]
trace0 = go.Scatter(
    x=x_data,
    y=y_data,
    mode='markers',
    marker=dict(
        size=[i * 2 for i in y_data],
    )
)

data = [trace0]
layout = go.Layout(annotations=annotations)

fig = go.Figure(data=data, layout=layout)
fig.show()

In [237]:
import spacy
nlp = spacy.load("en_core_web_sm")
# func to get the 'adjective&adverb', 'verb', 'noun' in each review and store them into list
def pos_func(text):
    adj_adv = []
    verb = []
    noun = []
    propn = []
    combined_doc = nlp(text)
    for token in combined_doc:
        if token.pos_ == 'ADJ' or token.pos_ == 'ADV':
            if not str(token) == '-':
                adj_adv.append(str(token))
        elif token.pos_ == 'VERB':
            verb.append(str(token))
        elif token.pos_ == 'NOUN':
            noun.append(str(token))
        else:
            continue
    return adj_adv, verb, noun

In [238]:
# func to draw frequency bubble chart for each pos_tag word list
def frequency_bubble_chart(pos_tag_list,count,x = 1):
    from collections import Counter
    word_freq = Counter(pos_tag_list)
    common_words = word_freq.most_common(count)
    ### Draw frequency bubble charts
    import plotly.graph_objects as go
    x_data = list(range(count))
    y_data = list(zip(*common_words))[1]
    z_data = list(zip(*common_words))[0]
    annotations = [
        dict(
            x=x_data[i], 
            y=y_data[i],
            text=z_data[i], # Some conditional to define outliers
            showarrow=False,
            xanchor='center',  # Position of text relative to x axis (left/right/center)
            yanchor='middle',  # Position of text relative to y axis (top/bottom/middle)
        ) for i in range(count)
    ]
    trace0 = go.Scatter(
        x=x_data,
        y=y_data,
        mode='markers',
        marker=dict(
            size=[i*x for i in y_data],
        )
    )

    data = [trace0]
    layout = go.Layout(annotations=annotations)

    fig = go.Figure(data=data, layout=layout)
    fig.show()
    return common_words

In [239]:
delete_words = ['soap', 'grams', 'units', 'gramas', 'barra', 'bar', 'sabonetes', 'sabonete', 'g', 'soaps']
# func to remove unnecessary words
def remove_words(raw_text):
    texts = []
    stop_words = set(stopwords.words('english')) 
    for i in raw_text:
        if type(i) == str:
            word_tokens = word_tokenize(i.lower().strip().translate(str.maketrans('', '', string.punctuation)))
            # remove stop words and delete words
            stopped_tokens = [i for i in word_tokens if not i in stop_words.union(set(delete_words))]
            texts.append(' '.join(stopped_tokens))
        else:
            continue
    return texts

In [250]:
adj_adv, verb, noun = pos_func(' '.join(remove_words(model)))
pos_dict = {}
pos_dict['adj_adv'] = adj_adv
pos_dict['verb'] = verb
noun = [word for word in noun if word not in ['g','line','information','skin','body']]
pos_dict['noun'] = noun

In [251]:
count = 15
for key,value in pos_dict.items():
    if key == 'adj_adv':
        common_words = frequency_bubble_chart(value,count,3)
    elif key == 'verb':
        common_words = frequency_bubble_chart(value,10,3)
    else:
        common_words = frequency_bubble_chart(value,count,3)

## Description

In [422]:
from gensim.models.ldamodel import LdaModel
from gensim import corpora
def prepare_text(texts,delete_list):
    for i in range(len(texts)):
        text = texts[i]
        sents = sent_tokenize(text)
        for j in range(len(sents)):
            sent = sents[j]
            sent = sent.strip().replace('\n','')
            sents[j] = sent
        texts[i] = '. '.join(sents)
    texts = [[word for word in text.lower().split()
        if word not in STOPWORDS and word.isalnum() and not word.lower() in delete_list and len(word) > 3]
        for text in texts]
    return texts
def lda_func(texts,num_topics, passes, delete_list):
    texts = prepare_text(texts,delete_list)
    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]
    lda = LdaModel(corpus,
                  id2word=dictionary,
                  num_topics=num_topics,
                  passes=passes)
    return lda

In [ ]:
texts = [translator.translate(i).text for i in bar_soap_Brazil['description'].dropna()]

In [261]:
lda = lda_func(texts,5,50,[' '])
lda.print_topics(num_words=10)

[(0,
  '0.032*"soap" + 0.022*"skin" + 0.012*"clay" + 0.009*"bar" + 0.009*"dove" + 0.007*"natural" + 0.007*"oil" + 0.006*"sodium" + 0.006*"helps" + 0.006*"will"'),
 (1,
  '0.028*"soap" + 0.019*"skin" + 0.014*"bar" + 0.010*"natural" + 0.009*"soft" + 0.009*"soaps" + 0.009*"formula" + 0.008*"natura" + 0.007*"90g" + 0.006*"kit"'),
 (2,
  '0.035*"soap" + 0.032*"skin" + 0.016*"daily" + 0.016*"natural" + 0.013*"without" + 0.012*"antiseptic" + 0.009*"contains" + 0.009*"use" + 0.009*"oily" + 0.009*"indicated"'),
 (3,
  '0.037*"de" + 0.036*"e" + 0.027*"o" + 0.014*"da" + 0.014*"em" + 0.013*"para" + 0.010*"que" + 0.009*"é" + 0.007*"frete" + 0.007*"sabonete"'),
 (4,
  '0.022*"shipping" + 0.022*"products" + 0.017*"will" + 0.017*"product" + 0.016*"mercado" + 0.013*"free" + 0.012*"buy" + 0.012*"informed" + 0.010*"made" + 0.009*"sodium"')]

In [264]:
import gensim.summarization
keyword = gensim.summarization.keywords(text = ' '.join(texts), words = 20, lemmatize = True).split('\n')
keyword

['______________________________________________________________soap',
 'naturals',
 'bars',
 'oils',
 'fragrances',
 'g__________________________________________________description',
 '_______________________________________sabonete',
 'peles',
 'containers',
 'moisturized',
 'extraction',
 'uses',
 'uma',
 'vegetation',
 '_________________________________________________________suas',
 'cleanness',
 'actions',
 'descriptionthe',
 'kit',
 'hydratant']

In [274]:
delete_words = ['soap', 'grams', 'units', 'gramas', 'barra', 'male', 'mercado', 'pm', 'skin', 'full', 'w3', 'product', 'zip', 'loccitane', 'shopping', 'service', 'use', 'purchase', 'shipping', 'delivery']
import spacy
# set up nlp environment in spacy
nlp = spacy.load("en_core_web_sm")
# use spacy to identify each chunk in the review after covid and put them into the chunk list
chunk_list = []
for text in texts:
    doc = nlp(text.lower())
    for chunk in doc.noun_chunks:
        chunk = ' '.join([word for word in str(chunk.text).split() if word not in STOPWORDS])
        if not (len(chunk.split(' ')) == 0 or len(chunk.split(' ')) == 1):
            chunk_list.append(chunk)
# create chunk dict to record each chunk's appearence time
chunk_dict = dict()
for i in chunk_list:
    if not set.intersection(set(i.lower().split(' ')), set(delete_words)):
        if chunk_dict.get(i):
            chunk_dict[i] += 1
        else:
            chunk_dict[i] = 1
# sort chunk dict based on the value
sort_list = sorted(chunk_dict.items(), key=lambda x: x[1], reverse=True)

In [275]:
sort_list

[('creamy foam', 134),
 ('sodium hydroxide', 129),
 ('sodium chloride', 104),
 ('etidronic acid', 101),
 ('natura express', 95),
 ('titanium dioxide', 88),
 ('sodium palmate', 87),
 ('sodium palm kernelate', 81),
 ('pleasant sensation', 74),
 ('sodium stearate', 70),
 ('gentle formula', 70),
 ('excess oil', 68),
 ('original products', 60),
 ('tracking code', 58),
 ('next business day', 58),
 ('natural ingredients', 54),
 ('free market', 54),
 ('animal origin', 53),
 ('tetrasodium edta', 52),
 ('natural hydration', 51),
 ('immediate dispatch', 51),
 ('leading platinum market', 51),
 ('authorized reseller', 51),
 ('fiscal note', 51),
 ('better country', 51),
 ('green reputation bar', 51),
 ('right side', 51),
 ('several variables', 51),
 ('technical error', 51),
 ('several different products', 51),
 ('desired products', 51),
 ('search field', 51),
 ('different places', 51),
 ('cpf / cnpj', 51),
 ('mineral oil', 49),
 ('natural extracts', 48),
 ('natura consultant', 47),
 ('dead cells', 4

In [276]:
# draw bubble chart for the most ten frequency chunks
import plotly.graph_objects as go
x_data = list(range(10))
y_data = list(zip(*sort_list))[1][:10]
z_data = list(zip(*sort_list))[0][:10]
annotations = [
    dict(
        x=x_data[i], 
        y=y_data[i],
        text=z_data[i], # Some conditional to define outliers
        showarrow=False,
        xanchor='center',  # Position of text relative to x axis (left/right/center)
        yanchor='top' if i%2 else 'bottom',  # Position of text relative to y axis (top/bottom/middle)
    ) for i in range(10)
]
trace0 = go.Scatter(
    x=x_data,
    y=y_data,
    mode='markers',
    marker=dict(
        size=[i * 0.5 for i in y_data],
    )
)

data = [trace0]
layout = go.Layout(annotations=annotations)

fig = go.Figure(data=data, layout=layout)
fig.show()

In [287]:
nlp.max_length = 1230000
adj_adv, verb, noun = pos_func(' '.join(remove_words(texts)))
pos_dict = {}
adj_adv = [word for word in adj_adv if word not in ['free','daily','best','always','also','new','high']]
pos_dict['adj_adv'] = adj_adv
verb = [word for word in verb if word not in ['helps','made','contains','leaving','used','apply','indicated','g']]
pos_dict['verb'] = verb
noun = [word for word in noun if word not in ['g','products','bar','information','skin','body','foams','soap','formula','ingredients','day','water']]
pos_dict['noun'] = noun

In [288]:
count = 10
for key,value in pos_dict.items():
    if key == 'adj_adv':
        common_words = frequency_bubble_chart(value,count,0.1)
    elif key == 'verb':
        common_words = frequency_bubble_chart(value,count,0.1)
    else:
        common_words = frequency_bubble_chart(value,count,0.1)

# Amazon Scraping

In [352]:
page_url = ['https://www.amazon.com.br/s?k=sabonetes+em+barra&__mk_pt_BR=%C3%85M%C3%85%C5%BD%C3%95%C3%91&qid=1601827782&ref=sr_pg_1',
            'https://www.amazon.com.br/s?k=sabonetes+em+barra&page=2&__mk_pt_BR=%C3%85M%C3%85%C5%BD%C3%95%C3%91&qid=1601827846&ref=sr_pg_2',
            'https://www.amazon.com.br/s?k=sabonetes+em+barra&page=3&__mk_pt_BR=%C3%85M%C3%85%C5%BD%C3%95%C3%91&qid=1601827869&ref=sr_pg_3',
            'https://www.amazon.com.br/s?k=sabonetes+em+barra&page=4&__mk_pt_BR=%C3%85M%C3%85%C5%BD%C3%95%C3%91&qid=1601827895&ref=sr_pg_4',
            'https://www.amazon.com.br/s?k=sabonetes+em+barra&page=5&__mk_pt_BR=%C3%85M%C3%85%C5%BD%C3%95%C3%91&qid=1601827908&ref=sr_pg_5',
            'https://www.amazon.com.br/s?k=sabonetes+em+barra&page=6&__mk_pt_BR=%C3%85M%C3%85%C5%BD%C3%95%C3%91&qid=1601827921&ref=sr_pg_6',
            'https://www.amazon.com.br/s?k=sabonetes+em+barra&page=7&__mk_pt_BR=%C3%85M%C3%85%C5%BD%C3%95%C3%91&qid=1601827938&ref=sr_pg_7']

In [353]:
f1 = open("productUrl.txt", "x")
f2 = open("productReviewUrl.txt", "x")

In [354]:
for url in page_url:
    driver.get(url)
    for i in driver.find_elements_by_xpath("//span[@class='rush-component']/a"):
        f1 = open("productUrl.txt", "a")
        f1.write(i.get_attribute('href') + '\n')
f1.close()

In [358]:
f = open("productUrl.txt", "r")
product_urls = f.read()

In [359]:
product_urls_list =  product_urls.split('\n')
product_urls_list

['https://www.amazon.com.br/Sabonete-Palmolive-Naturals-Segredo-Sedutor/dp/B07Q4QTLMV/ref=sr_1_1?__mk_pt_BR=%C3%85M%C3%85%C5%BD%C3%95%C3%91&dchild=1&keywords=sabonetes+em+barra&qid=1601827782&sr=8-1',
 'https://www.amazon.com.br/Sabonete-Barra-Am%C3%AAndoas-80g-Johnsons/dp/B07W7W2RML/ref=sr_1_2?__mk_pt_BR=%C3%85M%C3%85%C5%BD%C3%95%C3%91&dchild=1&keywords=sabonetes+em+barra&qid=1601827782&sr=8-2',
 'https://www.amazon.com.br/Sabonete-Protex-Balance-Saud%C3%A1vel-Desconto/dp/B07L5WDV9B/ref=sr_1_3?__mk_pt_BR=%C3%85M%C3%85%C5%BD%C3%95%C3%91&dchild=1&keywords=sabonetes+em+barra&qid=1601827782&sr=8-3',
 'https://www.amazon.com.br/Sabonete-Palmolive-Naturals-Hidrata%C3%A7%C3%A3o-Saud%C3%A1vel/dp/B07LFLWD12/ref=sr_1_4?__mk_pt_BR=%C3%85M%C3%85%C5%BD%C3%95%C3%91&dchild=1&keywords=sabonetes+em+barra&qid=1601827782&sr=8-4',
 'https://www.amazon.com.br/Framboesa-Orqu%C3%ADdea-Sabonetes-Barra-Organica/dp/B07FFSQCNP/ref=sr_1_5?__mk_pt_BR=%C3%85M%C3%85%C5%BD%C3%95%C3%91&dchild=1&keywords=sabonetes+em+

In [365]:
for url in product_urls_list:
    try:
        driver.get(url)
        f2 = open("productReviewUrl.txt", "a")
        f2.write(driver.find_elements_by_xpath("//a[@data-hook='see-all-reviews-link-foot']")[0].get_attribute('href') + '\n')
    except:
        pass
f2.close()

In [373]:
def get_info_amazon(product_url):
    # get the basic info for the product
    driver.get(product_url)
    dict_ = dict()
    dict_['name'] = translator.translate(driver.find_elements_by_xpath("//span[@id='productTitle']")[0].text.strip()).text
    dict_['url'] = product_url
    try:
        dict_['price'] = float(driver.find_elements_by_xpath("//span[@id='priceblock_ourprice']")[0].text.split('$')[1])
    except:
        pass
    try:
        dict_['small_desp'] = translator.translate(driver.find_elements_by_xpath("//ul[@class='a-unordered-list a-vertical a-spacing-mini']")[0].text.strip()).text
    except:
        pass
    try:
        general_info = dict()
        for item in driver.find_elements_by_xpath("//table[@id='productDetails_techSpec_section_1']/tbody/tr"):
            general_info[translator.translate(item.find_elements_by_xpath(".//th")[0].text).text] = translator.translate(item.find_elements_by_xpath(".//td")[0].text).text
        dict_.update(general_info) 
    except:
        pass
    try:
        other_info = dict()
        for item in driver.find_elements_by_xpath("//table[@id='productDetails_detailBullets_sections1']/tbody/tr"):
            other_info[translator.translate(item.find_elements_by_xpath(".//th")[0].text).text] = translator.translate(item.find_elements_by_xpath(".//td")[0].text).text
        dict_.update(other_info)
    except:
        pass
    try:
        dict_['description'] = translator.translate(driver.find_elements_by_xpath("//div[@id='productDescription']")[0].text).text
    except:
        pass
    try:
        key = translator.translate(driver.find_elements_by_xpath("//div[@class='a-section content']")[0].text).text.split('\n')[0]
        value = translator.translate(driver.find_elements_by_xpath("//div[@class='a-section content']")[0].text).text.split('\n')[1]
        dict_[key] = value
    except:
        pass
    try:
        key = translator.translate(driver.find_elements_by_xpath("//div[@class='a-section content']")[1].text).text.split('\n')[0]
        value = translator.translate(driver.find_elements_by_xpath("//div[@class='a-section content']")[1].text).text.split('\n')[1]
        dict_[key] = value
    except:
        pass
    return dict_

In [302]:
# setting up the chrome driver
driver = webdriver.Chrome('/Users/guopinjie/Downloads/chromedriver')

In [374]:
amazon_products = []

In [379]:
for url in product_urls_list:
    amazon_products.append(get_info_amazon(url))

In [382]:
amazon_products_Brazil = pd.DataFrame(amazon_products)

In [413]:
amazon_products_Brazil

,name,url,small_desp,Volume,Units,Brand,Format,Number of units,Size,Department,...,ASIN,Does the battery or battery work?,If,Package dimensions,Available for purchase from,Customer reviews,Top sellers ranking,description,Indications,Ingredientes
0,Palmolive Naturals Soap Bar Soap Secret 85G Li...,https://www.amazon.com.br/Sabonete-Palmolive-N...,Made with 100% natural ingredients\nBrings a f...,527 Milliliters,6 Gram,Palmolive,Bar,6,85g,Adults,...,Б074ТЛМВ,No,7891024042267,17 x 8.4 x 5 cm,"April 1, 2018",224 ratings\n4.7 out of 5 stars,Nº 36 in Beauty (Meet the Top 100 in the Beaut...,With extracts of Raspberry and Tourmaline. Ins...,No,Sodium Oleate / Sodium Palmitate / Sodium Laur...
1,"Johnson's Almond Bar Soap 80g, Johnson's - L6P...",https://www.amazon.com.br/Sabonete-Barra-Am%C3...,Cleans and moisturizes the skin without leavin...,NaN,6 Unit,Johnson's,Bar,NaN,80g,NaN,...,B07 and Hawarmel,No,7891010247416,14 x 11 x 5.4 cm,7 august 2019,52 ratings\n4.7 out of 5 stars,Nº 165 in Beauty (Meet the Top 100 in the Beau...,johnson's almond soap 80 grs was developed for...,No,NaN
2,"Protex Healthy Balance Bar Soap (6x85g), 510g,...",https://www.amazon.com.br/Sabonete-Protex-Bala...,Size: 85 g\nEliminates 99.9% of bacteria\nExcl...,NaN,510 Gram,Protex,Bar,6,85G,Adults,...,B07 Lakhudfsab,No,7891024035559,16.4 x 8 x 3.6 cm,21 janeiro 2019,478 ratings\n4.8 out of 5 stars,Nº 135 in Beauty (Meet the Top 100 in the Beau...,Protex Balance offers a perfect balance betwee...,No,Sodium Oleate / Sodium Palmitate / Sodium Laur...
3,Palmolive Naturals Bar Soap Healthy Hydration ...,https://www.amazon.com.br/Sabonete-Palmolive-N...,Smooth and radiant skin\nAloe vera has healing...,527 Milliliters,6 Gram,Palmolive,Bar,6,85g,Adults,...,B07 for Flood 12,No,7891024039243,17 x 8.8 x 5.2 cm,17 December 2018,41 ratings\n4.6 out of 5 stars,Nº 999 in Beauty (Meet the Top 100 in the Beau...,"Combination of Aloe Vera and Olive Oil, 100% n...",No,Sodium Oleate / Sodium Palmitate / Sodium Laur...
4,Organic Raspberry And Orchid Soap Bar Kit 3X90...,https://www.amazon.com.br/Framboesa-Orqu%C3%AD...,We don't test on animals\nAll our cosmetics ar...,NaN,NaN,Organic,NaN,3,3 x x 0 c,Women,...,B07FFSQCNP,No,7896240209906,14.4 x 8.2 x 3.2 cm,21 janeiro 2019,7 ratings\n3.7 out of 5 stars,"Ranked # 4,097 in Beauty (Meet the Top 100 in ...",Scent: Raspberry and Orchid | Size: 3x90g\nKit...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,Protex Intimate Liquid Soap Delicate Care Inti...,https://www.amazon.com.br/Sabonete-%C3%8Dntimo...,Feminine liquid soap with a formula that helps...,200 Milliliters,NaN,NaN,Líquido,6,NaN,NaN,...,C07,NaN,7891024024003,NaN,21 janeiro 2019,"166 classificações\n4,8 de 5 estrelas",Ranked # 295 in Beauty (Meet the Top 100 in th...,Intimate Protex is a product developed especia...,No,NaN
302,"Johnson's Baby Infant Shampoo Prolonged, 200ml",https://www.amazon.com.br/Shampoo-Cheirinho-Pr...,Exclusive formula arrives with tears that don'...,NaN,NaN,Johnson's Baby,NaN,NaN,NaN,Crianças,...,B077 Saan,NaN,NaN,15.2 x 8.2 x 4.2 centímetros,NaN,64 ratings\n4.7 out of 5 stars,NaN,Size: 200ml\nThe johnson's shampoo 200ml prolo...,Não,NaN
303,"Sabonete Líquido HORTELÃ COM ERVA DOCE 500 ml,...",https://www.amazon.com.br/Fiorucci-Sabonete-L%...,Cleaning\nFreshness\nSoftness,NaN,1 Unidade,NaN,Líquido,NaN,NaN,NaN,...,B078B1VTBX,NaN,NaN,17.6 x 7.6 x 7.6 centímetros,NaN,"16 classificações\n4,8 de 5 estrelas","Nº 2,122 in Beauty (Meet the Top 100 in the Be...",O sabonete líquido fiorucci é enriquecido com ...,No,NaN
304,Hemoton 2 Unidades de Saboneteira Em Forma de ...,https://www.amazon.com.br/Hemoton-Saboneteira-...,"Drain the residual moisture from the soap, kee...",NaN,NaN,Hemoton,NaN,NaN,13X8.5CM,NaN,...,B08KGGK5WR,NaN,NaN,NaN,30 September 2020,NaN,NaN,CaracterÃ­sticas\n\n- Material PlÃ¡stico.\n - ...,NaN,NaN


In [414]:
amazon_products_Brazil.to_csv('amazon_products_Brazil.csv')

In [415]:
amazon_products_Brazil.columns

Index(['name', 'url', 'small_desp', 'Volume', 'Units', 'Brand', 'Format',
       'Number of units', 'Size', 'Department', 'Maker', 'Product dimensions',
       'ASIN', 'Does the battery or battery work?', 'If', 'Package dimensions',
       'Available for purchase from', 'Customer reviews',
       'Top sellers ranking', 'description', 'Indications', 'Ingredientes'],
      dtype='object')

# Analysis

In [416]:
# mercadolivre
model = [translator.translate(i).text for i in bar_soap_Brazil['Model'].dropna()]
texts = [translator.translate(i).text for i in bar_soap_Brazil['description'].dropna()]
# amazon
small_desp = [translator.translate(i).text for i in amazon_products_Brazil['small_desp'].dropna()]
description = [translator.translate(i).text for i in amazon_products_Brazil['description'].dropna()]

In [417]:
desp = model + texts + small_desp + description

In [418]:
desp

['Bouquet of Jasmine',
 'Original',
 'Bouquet of Jasmine',
 'Original',
 'Deep Cleaning',
 'Deep Cleaning',
 'Soap Floral',
 'Original',
 'Bar',
 'bar',
 '13 Grams',
 'Delicious',
 'Natural Soap',
 'Soap Floral',
 'Mother and baby',
 'VEGAN LINE',
 '0',
 'Bar',
 'Balanced Hydration',
 'Soap',
 'Bar',
 '13 Grams',
 'Mother and baby',
 'VEGAN LINE',
 'VEGAN LINE',
 'Perfumery Phebo Lavender - Bar Soap 150g',
 'VEGAN LINE',
 'Phebo Mediterranean Lavender Provençal - Bar Soap 100g',
 'Vegan Conditioner',
 'VEGAN LINE',
 'bar',
 'bar',
 '20 Grams',
 'bar',
 'bar',
 'bar',
 'Soap',
 'No information',
 'Baby Original',
 'bar',
 'bar',
 'Bar',
 'VEGAN LINE',
 'Tonic',
 'Glycerin',
 'Soap Floral',
 'VEGAN LINE',
 'bar',
 'bar',
 'Men Scented Soaps, 2 Units',
 'bar',
 'bar',
 'bar',
 'bar',
 'BAR SOAP',
 'bar',
 'VEGAN LINE BIBLICAL ARVORES',
 'No information',
 'Kinelyn',
 'bar',
 'Deep Cleaning',
 'Bar soap',
 'In Barra',
 'Vegetable Soap',
 'Deep Cleaning',
 'Fennel',
 'VEGAN FLOWERS LINE',
 

In [423]:
lda = lda_func(desp,5,50,['description','composition','action','actives','important','notice','bar','soap','grams','products','soaps','product','products','will','buy','skin'])
lda.print_topics(num_words=20)

[(0,
  '0.025*"para" + 0.019*"sabonete" + 0.018*"pele" + 0.010*"natura" + 0.010*"mercado" + 0.008*"barra" + 0.007*"natural" + 0.007*"produtos" + 0.005*"mais" + 0.005*"todos" + 0.005*"fórmula" + 0.005*"frete" + 0.005*"você" + 0.004*"limpa" + 0.004*"sabonetes" + 0.004*"express" + 0.004*"produto" + 0.004*"possui" + 0.004*"limpeza" + 0.004*"bath"'),
 (1,
  '0.019*"shipping" + 0.013*"mercado" + 0.012*"free" + 0.009*"sodium" + 0.009*"purchase" + 0.008*"informed" + 0.008*"delivery" + 0.008*"always" + 0.008*"made" + 0.008*"questions" + 0.007*"best" + 0.007*"sent" + 0.007*"prompt" + 0.007*"livre" + 0.007*"check" + 0.006*"immediate" + 0.006*"freight" + 0.006*"different" + 0.006*"full" + 0.006*"several"'),
 (2,
  '0.018*"natural" + 0.017*"sodium" + 0.012*"helps" + 0.011*"dove" + 0.009*"tested" + 0.009*"without" + 0.008*"free" + 0.008*"base" + 0.008*"artificial" + 0.007*"ingredients" + 0.007*"shipping" + 0.007*"original" + 0.007*"organic" + 0.007*"high" + 0.006*"formula" + 0.006*"protex" + 0.006*"

In [428]:
import gensim.summarization
prepare_desp = [' '.join(i) for i in prepare_text(desp,['description','composition','action','actives',
                       'important','notice','bar','soap','grams','products',
                       'soaps','product','products','will','buy','skin','mercado'])]
keyword = gensim.summarization.keywords(text = ' '.join(prepare_desp), words = 50, lemmatize = True).split('\n')
keyword

['para',
 'naturals',
 'peles',
 'vegetation',
 'formulas',
 'unites',
 'moisturized',
 'containers',
 'produto',
 'cleanness',
 'fragrances',
 'bodies',
 'sodium',
 'perfumers',
 'soft',
 'free',
 'hydratant',
 'extraction',
 'ideal',
 'originals',
 'bases',
 'protects',
 'creaminess',
 'feels',
 'careful',
 'fragrancias',
 'essentially',
 'beautiful',
 'daily',
 'combine',
 'promotions',
 'barras',
 'remover',
 'aromas',
 'glycerinated',
 'oiliness',
 'babys',
 'oleos',
 'bathe',
 'lines',
 'tests',
 'liquid',
 'additives',
 'specialized',
 'shipped',
 'todo',
 'facial',
 'possui',
 'dove',
 'delicately']

In [429]:
delete_words = ['soap', 'grams', 'units', 'gramas', 'barra', 'male', 'mercado', 'pm', 'skin', 'full', 'w3', 'product', 'zip', 'loccitane', 'shopping', 'service', 'use', 'purchase', 'shipping', 'delivery']
import spacy
# set up nlp environment in spacy
nlp = spacy.load("en_core_web_sm")
# use spacy to identify each chunk in the review after covid and put them into the chunk list
chunk_list = []
for text in desp:
    doc = nlp(text.lower())
    for chunk in doc.noun_chunks:
        chunk = ' '.join([word for word in str(chunk.text).split() if word not in STOPWORDS 
                         and word.isalnum() and len(word) > 3])
        if not (len(chunk.split(' ')) == 0 or len(chunk.split(' ')) == 1):
            chunk_list.append(chunk)
# create chunk dict to record each chunk's appearence time
chunk_dict = dict()
for i in chunk_list:
    if not set.intersection(set(i.lower().split(' ')), set(delete_words)):
        if chunk_dict.get(i):
            chunk_dict[i] += 1
        else:
            chunk_dict[i] = 1
# sort chunk dict based on the value
sort_list = sorted(chunk_dict.items(), key=lambda x: x[1], reverse=True)

In [430]:
sort_list

[('creamy foam', 142),
 ('sodium hydroxide', 128),
 ('etidronic acid', 104),
 ('sodium chloride', 102),
 ('natura express', 96),
 ('titanium dioxide', 89),
 ('sodium palmate', 87),
 ('natural hydration', 84),
 ('sodium palm kernelate', 81),
 ('natural ingredients', 81),
 ('animal origin', 79),
 ('gentle formula', 75),
 ('pleasant sensation', 73),
 ('sodium stearate', 69),
 ('cool place', 61),
 ('original products', 60),
 ('mineral oils', 58),
 ('natural origin', 56),
 ('desired region', 56),
 ('moisturizing cream', 55),
 ('shea butter', 55),
 ('next business', 55),
 ('tracking code', 54),
 ('vegetable base', 54),
 ('natural moisture', 53),
 ('vegetable oils', 53),
 ('salicylic acid', 53),
 ('deep cleansing', 52),
 ('high durabilityfragrance', 52),
 ('protective barrier', 51),
 ('tetrasodium edta', 50),
 ('sealed products', 48),
 ('dead cells', 48),
 ('immediate dispatch', 48),
 ('leading platinum market', 48),
 ('authorized reseller', 48),
 ('fiscal note', 48),
 ('better country', 48),

In [431]:
# draw bubble chart for the most ten frequency chunks
import plotly.graph_objects as go
x_data = list(range(10))
y_data = list(zip(*sort_list))[1][:10]
z_data = list(zip(*sort_list))[0][:10]
annotations = [
    dict(
        x=x_data[i], 
        y=y_data[i],
        text=z_data[i], # Some conditional to define outliers
        showarrow=False,
        xanchor='center',  # Position of text relative to x axis (left/right/center)
        yanchor='top' if i%2 else 'bottom',  # Position of text relative to y axis (top/bottom/middle)
    ) for i in range(10)
]
trace0 = go.Scatter(
    x=x_data,
    y=y_data,
    mode='markers',
    marker=dict(
        size=[i * 0.5 for i in y_data],
    )
)

data = [trace0]
layout = go.Layout(annotations=annotations)

fig = go.Figure(data=data, layout=layout)
fig.show()

In [453]:
nlp.max_length = 1300000
adj_adv, verb, noun = pos_func(' '.join(remove_words(desp)))
pos_dict = {}
adj_adv = [word for word in adj_adv if word not in ['free','daily','best','always','also','new','high'] 
           and word not in STOPWORDS and word.isalnum() and len(word) > 3]
pos_dict['adj_adv'] = adj_adv
verb = [word for word in verb if word not in ['helps','made','contains','leaving','used','apply','indicated','g','provides','keeps','leaves','rinse','using','leave','provide','developed','make','take','providing','keep','cleansing','cleaning','desired','sent','receive']
       and word not in STOPWORDS and word.isalnum() and len(word) > 3]
pos_dict['verb'] = verb
noun = [word for word in noun if word not in ['g','products','bar','information','skin','body','foams','soap','formula','ingredients','day','water','action','rinse','bath','addition','properties','invoice','types','soaps']
       and word not in STOPWORDS and word.isalnum() and len(word) > 3]
pos_dict['noun'] = noun

In [454]:
count = 10
for key,value in pos_dict.items():
    if key == 'adj_adv':
        common_words = frequency_bubble_chart(value,count,0.1)
    elif key == 'verb':
        common_words = frequency_bubble_chart(value,count,0.1)
    else:
        common_words = frequency_bubble_chart(value,count,0.1)